In [1]:
#====================================================================================================
# kogpt2 를 이용해 훈련한 모델 추론(생성) 요약(abstractive summarization) 테스트 예시
# => 
# => https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/
#
# [테스트 데이터]
# => input_ids = 요약할 문장<segment>
# 여기서 테스트 모델은 kogpt2 pretrain 모델에, '요약할문장<segment>요약문장</s>' 식에 데이터 셋을 가지고,
# fine-tuning한 모델임. 따라서 반드시, '요약할 문장<segment>' 식으로 입력을 해야 함
# <segment> 토큰은 훈련할때 임의로 변경할수 있음.

# [추론(생성) 요약(abstractive summarization) 추론 과정]
# 
# 1. gpt-2 모델 선언(GPT2LMHeadModel), tokenizer 선언(PreTrainedTokenizerFast)
# 2. '요약할 문장+구분token(<summarize>) 를 모델에 입력
# 3. logits 얻음 : <summarize> 다음에 올 단어들의 확률분포(총 embedding 단어계수 배열이 출력됨)
# 4. top=k 알고리즘으로 logits중에서 확률이 높은 상위 k계 단어 중에서, 랜덤한 1개의 단어 선택
# 5. 선택한 단어를 다시 모델에 입력 
# 6. 출력된 logits에서 다시 확률이 가장높은 상위 k 단어중에서, 랜덤한 1개의 단어 선택
# 7 . 5.6 과정을 반복(단어의 계수가 max_len 이될때까지 혹은 eos 토큰이 출력될때까지)
#====================================================================================================

import torch
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler 
import numpy as np
import pandas as pd
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

from tqdm.notebook import tqdm
import os
import time
from myutils import GPU_info, seed_everything, mlogging

model_path='../model/gpt-2/kogpt-2-ft-summarizer-0509/'


device = GPU_info()
print(device)

#seed 설정
seed_everything(222)

#logging 설정
logger =  mlogging(loggername="gpt2-summarize-test", logfilename="../log/gpt2-summarize-test")

logfilepath:../log/bwdataset_2022-05-09.log
logfilepath:../log/qnadataset_2022-05-09.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../log/gpt2-summarize-test_2022-05-09.log


In [2]:
# tokenizer 로딩 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path,
                                                   bos_token='</s>',
                                                   eos_token='</s>',
                                                   unk_token='<unk>',
                                                   pad_token='<pad>',
                                                   mask_token='<mask>')
# 모델 로딩
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [6]:
# 상위 n 만큼 단어들을 선택함.
def topk(probs, n=9):
    probs = torch.softmax(probs, dim=-1)  # 0~1까지 모든 embedding 단어들의 확률값 구함
    
    # 상위 k 계수 단어들의 확률값을 구함
    tokensProb, topIx = torch.topk(probs, k=n)
    tokensProb = tokensProb / torch.sum(tokensProb)
    
    tokensProb = tokensProb.cpu().detach().numpy()
    
    # 랜덤하게 k 단어들에 대해 1개의 단어를 리턴 함
    choice = np.random.choice(n,1,p=tokensProb)
    tokenId = topIx[choice][0]
    
    return int(tokenId)

In [30]:
# summarize 추론 함수
def model_infer(model, 
                tokenizer, 
                review, 
                max_length=15,  # 최대 요약문 출력 단어수
                topk_num=9,     # 임베딩 중 상위 k 선택 항목수(*10으로 지정하면, 10개의 단어중 랜덤한 1단어가 다음 단어로 출력됨)
                eos_stop=True): # true = eos 토큰이 출력되면 요약문 생성 종료, false=max_length 만큼 무조건 요약문 생성
    
    # tokenizer 로 '요약할 문장<segment>' encode 하여 id 출력
    review_encoded = tokenizer.encode(review)
   
    # result = 요약할 문장<segment>
    result = review_encoded
    #print(f'result:{result}')
    print(f'result len:{len(result)}') 
    
    # tokenizer된 id를 1차원으로 만듬
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to(device)
    with torch.set_grad_enabled(False):
        
        # 1차원으로 된 요약할 문장<segment> token 모델에 입력
        outputs = model(initial_input)
        
        # outputs에는 embedding 계수가 출력됨
        logits = outputs.logits[0,-1]
        print(f'logits.shape:{logits.shape}')  
       
        # result = 요약할 문장<segment> + embedding 계수중 상위 topk_num 계중 랜덤한 1개 단어
        res_id = topk(logits, topk_num)
        print(f'res_id:{res_id}-{tokenizer.decode(res_id)}')
        result.append(res_id)
        
        #print(f'out: result:{result}')
        print(f'out: result len:{len(result)}')
     
        # max_length 만큼 돌면서, 
        for _ in range(max_length):
            # 앞에 result 값이 들어감
            input = torch.tensor(result).unsqueeze(0).to(device)
            #print(f'input:{input}')
                
            # 다음 단어 출력 
            outputs = model(input)
            logits = outputs.logits[0,-1]
            
            # 상위 topk_num 계중 랜덤한 1개 단어 선택
            res_id = topk(logits, topk_num)
            print(f'res_id:{res_id}-{tokenizer.decode(res_id)}')
            
            # EOS 토큰을 만날때까지 계속 max_length 만큼 돔
            if eos_stop:
                if res_id == tokenizer.eos_token_id:
                    return tokenizer.decode(result)
                else:
                    result.append(res_id)
            else:
                result.append(res_id)
            
    return tokenizer.decode(result)

In [62]:
# A review is initially fed to the model.
# A choice from the top-k choices is selected.
# The choice is added to the summary and the current sequence is fed to the model.
# Repeat steps 2 and 3 until either max_len is achieved or the EOS token is generated.

body = '''
청와대 개방을 맞아 청와대에서 시청역과 남산타워, 충무로역을 잇는 순환버스 노선이 신설됐다. 
청와대 진입로에 위치했던 검문소는 철거되고, 급경사가 많은 백악정~북악산 등산로 구간에는 데크와 계단이 설치된다. 
청와대 사랑채에서 시작하는 야간 경관 해설 프로그램 등 문화행사도 진행된다.
이달 10일 청와대 개방을 앞두고 서울시는 청와대 인근 교통체계 개편과 문화행사 기획을 담은 종합지원대책을 발표했다. 
청와대 개방 행사가 열리는 10일부터 22일까지 1일 평균 관광객이 4만명가량 증가할 것이라는 계산이다
'''

output = model_infer(model, tokenizer, body + "<summarize>", max_length = 20, topk_num=10, eos_stop=True)
#print(output)

# 출력 output에는 body<summarize>요약문 식으로 출력되므로, 여기서는 <summarize> 토크 다음 요약문만 출력 함
summary = output.split("<summarize>")[1].strip()
print(summary)

result len:131
logits.shape:torch.Size([51200])
res_id:8367-청
out: result len:132
res_id:8066-와
res_id:7198-대
res_id:20508-도심
res_id:14499-순환
res_id:28147-버스
res_id:31957-노선
res_id:11454-신설
res_id:739-
res_id:739-
res_id:739-
res_id:739-
res_id:26074-시내
res_id:12942-접근
res_id:7799-성
res_id:10416-강화
res_id:375-

res_id:1-</s>
청와대 도심 순환버스 노선 신설     시내 접근성 강화


In [53]:
print(tokenizer.decode(8187))
print(tokenizer.decode(10347))
print(tokenizer.decode(739))
print(tokenizer.decode(34327))

전
남도

신재
